# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import json 
from sql_queries import *
import datetime
import numpy as np

In [2]:
conn = psycopg2.connect("host=10.0.0.231 dbname=sparkifydb user=postgres password=dummypwd")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")
#Display all outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
fulldata = pd.DataFrame()
for file in song_files:
    with open(file) as f:
        data = json.load(f)
        fulldata = fulldata.append(data, ignore_index = True)

In [6]:
df = fulldata
df.head(10)

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR7G5I41187FB4CE6C,None,"London, England",None,Adam Ant,233.40363,1.0,SONHOTT12A8C13493C,Something Girls,1982.0
1,AR8ZCNI1187B9A069B,None,,None,Planet P Project,269.81832,1.0,SOIAZJW12AB01853F1,Pink World,1984.0
2,ARXR32B1187FB57099,None,,None,Gob,209.60608,1.0,SOFSOCN12A8C143F5D,Face the Ashes,2007.0
3,AR10USD1187B99F3F1,None,"Burlington, Ontario, Canada",None,Tweeterfriendly Music,189.57016,1.0,SOHKNRJ12A6701D1F8,Drop of Rain,0.0
4,ARGSJW91187B9B1D6B,35.2196,North Carolina,-80.0195,JennyAnyKind,218.77506,1.0,SOQHXMF12AB0182363,Young Boy Blues,0.0
5,ARMJAGH1187FB546F3,35.1497,"Memphis, TN",-90.0489,The Box Tops,148.03546,1.0,SOCIWDW12A8C13D406,Soul Deep,1969.0
6,ARD7TVE1187B99BFB1,None,California - LA,None,Casual,218.93179,1.0,SOMZWCG12A8C13C480,I Didn't Mean To,0.0
7,ARKRRTF1187B9984DA,None,,None,Sonora Santanera,177.47546,1.0,SOXVLOJ12AB0189215,Amor De Cabaret,0.0
8,ARNTLGG11E2835DDB9,None,,None,Clp,266.39628,1.0,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0.0
9,ARKFYS91187B98E58F,None,,None,Jeff And Sheri Easter,267.70240,1.0,SOYMRWW12A6D4FAB14,The Moon And I (Ordinary Day Album Version),0.0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
df = df.loc[:,['song_id','artist_id','title','year','duration']]
song_data = (df.values)

In [8]:
song_table_create
song_table_insert
type(song_data)

'\nCREATE TABLE IF NOT EXISTS songs (song_id varchar(40), title varchar(70), artist_id varchar(30) , year REAL, duration REAL)\n'

'INSERT INTO songs (song_id, artist_id, title, year, duration) VALUES (%s,%s,%s,%s,%s)\n'

numpy.ndarray

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
for song in song_data:
    cur.execute(song_table_insert, song)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
artist_data = fulldata
artist_data = artist_data.loc[:,['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]
artist_data = artist_data.values

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
for artist in artist_data:
    cur.execute(artist_table_insert, artist)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
log_files = get_files("data/log_data")

logdata = pd.DataFrame()
for file in log_files:
    df = pd.read_json(file, lines=True)
    logdata = logdata.append(df, ignore_index = True) 

In [13]:
df = logdata
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = logdata.loc[logdata['page']=='NextSong',:]
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [15]:
df.loc[:,'ts'] = pd.to_datetime(df['ts'],unit='ms')
df.head()

day = list(df['ts'].dt.day)
year = list(df['ts'].dt.year)
month = list(df['ts'].dt.month)
week = list(df['ts'].dt.week)
weekday = list(df['ts'].dt.weekday)
hour = list(df['ts'].dt.hour)
ts = list(df['ts'])

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,2018-11-11 02:33:56.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,2018-11-11 02:36:10.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,2018-11-11 02:40:34.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,2018-11-11 04:36:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,2018-11-11 04:36:46.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [16]:
date_list = []

for i in range(0,len(ts)):
    templist = [ts[i],year[i],month[i],day[i],hour[i],week[i],weekday[i]]
    date_list.append(templist)

time_data = (date_list)
column_labels = ('timestamp','year','month','day','hour','week','weekday')

In [17]:
# datetime.datetime.strptime(str(x[0]), '%Y-%m-%d %H:%M:%S.%f')
# #x[0].year
# tempdt = datetime.datetime.fromtimestamp(x[0]/ 1e3)
# pd.to_datetime(tempdt)
# templist = [tempdt, tempdt.year,tempdt.month,tempdt.day,tempdt.hour,tempdt.isoweekday(),tempdt.date().isocalendar()[1]]
# templist

In [18]:
time_df = pd.DataFrame(data=(time_data),columns=column_labels)
time_df.head()


,timestamp,year,month,day,hour,week,weekday
0,2018-11-11 02:33:56.796,2018,11,11,2,45,6
1,2018-11-11 02:36:10.796,2018,11,11,2,45,6
2,2018-11-11 02:40:34.796,2018,11,11,2,45,6
3,2018-11-11 04:36:13.796,2018,11,11,4,45,6
4,2018-11-11 04:36:46.796,2018,11,11,4,45,6


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [19]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [20]:
user_df = logdata.loc[:,['userId','firstName','lastName','gender','level']]
user_df = user_df.drop_duplicates()
user_df = user_df.loc[user_df['userId']!="",:]
user_df.head()

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
3,32,Lily,Burns,F,free
5,75,Joseph,Gutierrez,M,free
6,92,Ryann,Smith,F,free
11,49,Chloe,Cuevas,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [21]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [22]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,2018-11-11 02:33:56.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,2018-11-11 02:36:10.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,2018-11-11 02:40:34.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,2018-11-11 04:36:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,2018-11-11 04:36:46.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [23]:
for index, row in df.head(10).iterrows():
    # get songid and artistid from song and artist tables
    cur.execute(song_select, {'songname':row.song, 'artistname':row.artist})
    cur.fetchall()
    cur.query

[]

b"SELECT s.song_id, a.artist_id FROM songs s INNER JOIN artists a ON a.artist_id = s.artist_id\nWHERE s.title = 'Fuck Kitty' AND a.name = 'Frumpies' \n"

[]

b"SELECT s.song_id, a.artist_id FROM songs s INNER JOIN artists a ON a.artist_id = s.artist_id\nWHERE s.title = 'By The Time This Night Is Over' AND a.name = 'Kenny G with Peabo Bryson' \n"

[]

...

b"SELECT s.song_id, a.artist_id FROM songs s INNER JOIN artists a ON a.artist_id = s.artist_id\nWHERE s.title = 'Cuentale' AND a.name = 'Don Omar' \n"

[]

b"SELECT s.song_id, a.artist_id FROM songs s INNER JOIN artists a ON a.artist_id = s.artist_id\nWHERE s.title = 'D''Evils' AND a.name = 'Jay-Z' \n"

[]

b"SELECT s.song_id, a.artist_id FROM songs s INNER JOIN artists a ON a.artist_id = s.artist_id\nWHERE s.title = 'Easily (Album Version)' AND a.name = 'Red Hot Chili Peppers' \n"

In [24]:
for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
    cur.execute(song_select, {'songname':row.song, 'artistname':row.artist, 'duration':row.length})
    results = cur.fetchone()
    #cur.query
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    # insert songplay record
#     songplay_data = ()
#     cur.execute(songplay_table_insert, songplay_data)
#     conn.commit()
    print(songid,artistid)

None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None


None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None


KeyboardInterrupt: 

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.